In [1]:
from typing import List
import jsonlines

from tqdm.notebook import tqdm
import numpy as np

import torchaudio

DATA_BASE_PATH = "/mnt/storage/186.복지 분야 콜센터 상담데이터"

TRAIN_PATH = f"{DATA_BASE_PATH}/train.jsonl"
DEV_PATH = f"{DATA_BASE_PATH}/dev.jsonl"

In [10]:
def get_wav_lens(wav_path: str):
    data, sample_rate = torchaudio.load(wav_path)
    
    len_data = data.size()[1]    
    t = len_data / sample_rate
    
    return t, len_data


def walk(file_path: str):
    max_time, max_len = 0, 0
    
    with jsonlines.open(file_path) as f:
        for i, line in enumerate(f.iter()):
            wav_path = line["audio_path"]
            t, len_data = get_wav_lens(wav_path)
            
            max_time = np.maximum(max_time, t)
            max_len = np.maximum(max_len, len_data)
            
            if i > 0 and i % 1000 == 0:
                print(f"{i} :: {max_time}\t{max_len}")
            
    return max_time, max_len

In [ ]:
train_max_time, train_max_len = walk(TRAIN_PATH)
print(f"TRAIN :: {train_max_time}, {train_max_len}")

dev_max_time, dev_max_len = walk(DEV_PATH)
print(f"DEV :: {dev_max_time}, {dev_max_len}")


In [6]:
def walk2(file_path: str):
    max_text_len, max_label_len = 0, 0
    max_text_len_idx, max_label_len_idx = 0, 0
    
    with jsonlines.open(file_path) as f:
        for i, line in enumerate(f.iter()):
            text, label = line["text"], line["label"]
            
            if max_text_len < len(text):
                max_text_len = len(text)
                max_text_len_idx = i
                
            if max_label_len < len(label):
                max_label_len = len(label)
                max_label_len_idx = i
            
            # if i > 0 and i % 1000 == 0:
            #     print(f"{i} :: {max_text_len}\t{max_label_len}")
            
    return max_text_len, max_text_len_idx, max_label_len, max_label_len_idx

In [7]:
train_res = walk2(TRAIN_PATH)
print(f"TRAIN :: {train_res}")

dev_res = walk2(DEV_PATH)
print(f"DEV :: {dev_res}")


TRAIN :: (124, 104181, 87, 104181)
DEV :: (93, 3312, 63, 22110)
